In [ ]:

import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model


lemmatizer = WordNetLemmatizer()

intents = json.loads(open('content.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word)  for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words= clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda  x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list


def get_response(intents_list,intents_json):
    tag= intents_list[0]['intent']
    list_of_intents =intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("|=============== Ask your any query about our college by virtual bot ================|")
while True:
    message = input("| You: ")
    if message == "bye" or message == "Goodbye":
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)
        print("|===================== The Program End here! =====================|")
        exit()

    else:
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)

|=============== Ask your any query about our college by virtual bot ================|
| You: hello
1/1 [==============================] - 0s 187ms/step
| Bot: Hello!
| You: who are you
1/1 [==============================] - 0s 46ms/step
| Bot: You can call me paxbot
| You: courses
1/1 [==============================] - 0s 42ms/step
| Bot: NSUT offers following branches B.Tech Programmes at the Undergraduate Level:B.Tech In Electronics And Communication Engineering B.Tech In Computer Engineering B.Tech In Computer Science And Engineering (With Specialisation In Artificial Lntelligence) B.Tech In Mathematics And Computing B.Tech In Electrical Engineering B.Tech In Instrumentation And Control Engineering B.Tech In Manufacturing Process And Automation Engineering B.Tech In Mechanical Engineering B.Tech In Information Technology B.Tech In Bio-Technology
| You: location
1/1 [==============================] - 0s 46ms/step
| Bot: Dwarka Sector-3, Dwarka, Delhi, 110078
| You: how long is the b